In [1]:
# Import libraries
library(shiny)
library(data.table)
library(randomForest)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.



In [2]:
# Training set (without removing the first column)
TrainSet <- read.csv("training.csv", header = TRUE)

# Ensure that columns are numeric before using them in sliders
TrainSet$Sepal.Length <- as.numeric(TrainSet$Sepal.Length)
TrainSet$Sepal.Width <- as.numeric(TrainSet$Sepal.Width)
TrainSet$Petal.Length <- as.numeric(TrainSet$Petal.Length)
TrainSet$Petal.Width <- as.numeric(TrainSet$Petal.Width)

In [3]:
# Define UI
ui <- pageWithSidebar(
  
  # Page header
  headerPanel('Iris Predictor'),
  
  # Input values
  sidebarPanel(
    HTML("<h3>Input parameters</h3>"),
    sliderInput("Sepal.Length", label = "Sepal Length", value = 5.0,
                min = 4.3, max = 7.9),  # Adjust these ranges based on your dataset
    sliderInput("Sepal.Width", label = "Sepal Width", value = 3.6,
                min = 2.0, max = 4.4),  # Adjust these ranges based on your dataset
    sliderInput("Petal.Length", label = "Petal Length", value = 1.4,
                min = 1.0, max = 6.9),  # Adjust these ranges based on your dataset
    sliderInput("Petal.Width", label = "Petal Width", value = 0.2,
                min = 0.1, max = 2.5),  # Adjust these ranges based on your dataset
    
    actionButton("submitbutton", "Submit", class = "btn btn-primary")
  ),
  
  mainPanel(
    tags$label(h3('Status/Output')), # Status/Output Text Box
    verbatimTextOutput('contents'),
    tableOutput('tabledata') # Prediction results table
    
  )
)

In [4]:
# Define server function
server <- function(input, output, session) {
  
  # Load the model inside the server function
  model <- readRDS("model.rds")
  
  # Input Data
  datasetInput <- reactive({  
    
    # Create a data frame from input values
    df <- data.frame(
      Sepal.Length = as.numeric(input$Sepal.Length),
      Sepal.Width = as.numeric(input$Sepal.Width),
      Petal.Length = as.numeric(input$Petal.Length),
      Petal.Width = as.numeric(input$Petal.Width),
      stringsAsFactors = FALSE
    )
    
    # Make predictions using the trained model
    Output <- data.frame(Prediction = predict(model, df), 
                         round(predict(model, df, type = "prob"), 3))
    print(Output)
    
    return(Output)
  })
  
  # Status/Output Text Box
  output$contents <- renderPrint({
    if (input$submitbutton > 0) { 
      isolate("Calculation complete.") 
    } else {
      return("Server is ready for calculation.")
    }
  })
  
  # Prediction results table
  output$tabledata <- renderTable({
    if (input$submitbutton > 0) { 
      isolate(datasetInput()) 
    } 
  })
  
}

In [ ]:
# Create the shiny app
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:6799

